# Readme

CS GO remains a very popular game for over 2 decades. Global Offensive has one of the most popular esport scenes in the world. The Global Offensive professional scene consists of leagues and tournaments hosted by third-party organizations, and Valve-sponsored tournaments known as Major Championships. Majors are considered the most prestigious tournaments in Counter-Strike circuit and have among the largest prize pools; originally announced at US 250,000, the prize pools for Majors have risen to USD 1,000,000 since MLG Columbus 2016. Like in physical sports, football, basketball etc., the betting on e-sport competitive events is becoming popular. This project is aimed for predicting outcomes of CS GO matches based on individual players' performance.

Files:
- CS GO prediction.ipynb - main notebook where the final output files were generated from
- CS GO playground for tests.ipynb - additional notebook to understand data and to play with ML models
- train.csv and players_feats.csv - data for training and crossvalidation
- test.csv - data for the test set

Output files:
These files includes the new column called 'who_win' for test.csv
- test_data_1
- test_data_2
- test_data_3

# Exploratory analysis

Since the data is generated in-game, it should not require too much processing. 'map_id' acts like a unique game identifier, hence, tables are sorted according to that attribute. CS GO is played 5 vs 5 players. The file 'players_fear.csv' contains information about individual players, their statistics over recent 6 months. 'team_id' uniquely identifies each team. Most of the time we have the same players in a team, however, sometimes the team might change its players. Some player statistics are given in absolute values, while some given in ratios. Some columns can be calculated from other columns. For example, kill-death ratio can be calculated from kill divided death. Feature engineering might be importan.
Some columns contain NaNs, but those are rare, so rows with NaNs are dropped.

## Import data

### Train and players info

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import data and sort by map_id
players_feats = pd.read_csv('players_feats.csv')
players_feats.sort_values(by='map_id', inplace=True)
train_data = pd.read_csv('train.csv')
train_data.sort_values(by='map_id', inplace=True)

In [3]:
print(players_feats.isnull().sum())

p1_id                                   0
p1_total_kills                          0
p1_headshots                            0
p1_total_deaths                         0
p1_kd_ratio                             0
                                       ..
p5_team_win_percent_after_first_kill    4
p5_first_kill_in_won_rounds             0
team_id                                 0
map_name                                0
map_id                                  0
Length: 128, dtype: int64


In [4]:
train_data.isnull().sum()

map_id      0
team1_id    0
team2_id    0
map_name    0
who_win     0
dtype: int64

In [5]:
players_feats

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
1152,922,213,41.3,236,0.90,73.2,6.8,13,335,0.64,...,69,24,20,1.20,0.95,62.5,9.3,4869,Vertigo,0
1153,7938,581,48.2,469,1.24,93.1,11.2,26,716,0.81,...,-17,33,25,1.32,0.94,84.8,12.2,11251,Vertigo,0
1424,12521,270,42.6,334,0.81,78.9,4.5,17,422,0.64,...,-50,26,33,0.79,0.86,76.9,11.8,8297,Nuke,1
1425,7716,231,51.9,208,1.11,70.6,4.0,13,334,0.69,...,6,3,2,1.50,1.07,66.7,15.4,7020,Nuke,1
1059,8528,211,39.8,180,1.17,86.0,6.1,11,276,0.76,...,-31,26,48,0.54,0.85,80.8,13.0,5752,Ancient,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,9115,234,53.8,263,0.89,64.3,2.9,15,387,0.60,...,27,29,23,1.26,1.04,58.6,11.3,4494,Nuke,740
30,7322,172,40.7,173,0.99,80.5,7.8,10,267,0.64,...,5,25,27,0.93,0.97,76.0,13.4,9565,Overpass,741
31,429,167,47.3,154,1.08,78.5,3.7,8,227,0.74,...,31,17,8,2.12,1.02,82.4,12.6,6667,Overpass,741
10,429,252,47.6,264,0.95,79.9,7.7,14,378,0.67,...,45,29,16,1.81,0.98,75.9,10.6,6667,Inferno,742


Some columns contain NaNs, but those are rare, so rows with NaNs are dropped.

In [6]:
players_feats.dropna(axis=0, inplace=True)
players_feats.shape

(1470, 128)

### Exam data

In [7]:
# test data will be renamed into exam data
exam_data = pd.read_csv('test.csv')
exam_data.sort_values(by='map_id', inplace=True)
exam_data

,index,map_id,team1_id,team2_id,map_name
2,715,1,8297,7020,Nuke
17,730,45,4494,6665,Ancient
5,718,85,4494,4411,Vertigo
23,736,86,6667,7718,Ancient
8,721,125,4608,7718,Inferno
14,727,143,5973,8297,Mirage
26,739,201,4608,6667,Overpass
13,726,216,5973,8297,Ancient
19,732,220,7718,5973,Inferno
9,722,241,4608,7718,Mirage


## map_id as a game identifier

In [8]:
players_feats[['team_id','map_name','map_id']].head(10)

,team_id,map_name,map_id
1152,4869,Vertigo,0
1153,11251,Vertigo,0
1424,8297,Nuke,1
1425,7020,Nuke,1
1059,5752,Ancient,2
1058,4869,Ancient,2
465,11595,Dust2,3
464,10567,Dust2,3
667,4773,Vertigo,4
666,8297,Vertigo,4


In [9]:
train_data.head(10)

,map_id,team1_id,team2_id,map_name,who_win
78,0,4869,11251,Vertigo,0
173,2,4869,5752,Ancient,1
294,3,10567,11595,Dust2,1
435,4,8297,4773,Vertigo,1
395,5,7020,4869,Mirage,1
131,6,4608,7532,Dust2,0
468,7,10831,11501,Overpass,1
180,8,4494,10831,Vertigo,0
270,9,7532,6667,Nuke,1
656,10,4411,10426,Ancient,0


## 'team id'

In [10]:
temp_df = players_feats.sort_values(by=['team_id'], inplace=False)
temp_df[['p1_id','p2_id','p3_id', 'p4_id', 'p5_id','team_id']].head(10)

,p1_id,p2_id,p3_id,p4_id,p5_id,team_id
812,8611,9278,9766,13666,14218,4411
161,8611,9278,9766,14218,16726,4411
1010,8611,9278,13666,14218,16486,4411
867,8611,9278,9766,13666,14218,4411
478,8611,9278,9766,13666,14218,4411
159,8611,9278,9766,14218,16726,4411
85,8611,9278,9766,14218,16726,4411
1416,8611,9278,9766,13666,14218,4411
1433,8611,9278,9766,13666,14218,4411
239,8611,9278,9766,14218,16726,4411


## Player stats

For unknown reasons, for some players their statistics equal to zero.

In [11]:
players_feats.iloc[:,0:25]

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p1_kill_death,p1_kill_round,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds
1152,922,213,41.3,236,0.90,73.2,6.8,13,335,0.64,...,0.90,0.64,146,-23,34,49,0.69,0.93,67.6,12.9
1153,7938,581,48.2,469,1.24,93.1,11.2,26,716,0.81,...,1.22,0.83,295,80,91,55,1.65,1.35,70.3,23.8
1424,12521,270,42.6,334,0.81,78.9,4.5,17,422,0.64,...,0.78,0.63,163,-69,61,78,0.78,1.12,62.3,22.4
1425,7716,231,51.9,208,1.11,70.6,4.0,13,334,0.69,...,1.12,0.68,133,20,20,23,0.87,0.88,65.0,8.1
1059,8528,211,39.8,180,1.17,86.0,6.1,11,276,0.76,...,1.17,0.76,143,31,42,43,0.98,1.17,73.8,19.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,9115,234,53.8,263,0.89,64.3,2.9,15,387,0.60,...,0.89,0.60,169,-29,21,38,0.55,0.77,57.1,5.7
30,7322,172,40.7,173,0.99,80.5,7.8,10,267,0.64,...,0.99,0.64,112,-1,21,22,0.95,0.92,81.0,12.0
31,429,167,47.3,154,1.08,78.5,3.7,8,227,0.74,...,1.10,0.76,99,14,23,32,0.72,0.97,69.6,14.4
10,429,252,47.6,264,0.95,79.9,7.7,14,378,0.67,...,0.95,0.67,167,-12,63,54,1.17,1.26,76.2,23.2


### Some stats are missing

In [12]:
# shows that some players do not have their own stats while teammates have
players_feats[players_feats['p1_rounds_played'] == 0]

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
278,7168,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,54,22,9,2.44,1.17,81.8,16.4,4494,Vertigo,8
1242,5794,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,0,0,0,0.00,0.00,0.0,0.0,7532,Inferno,33
116,7168,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,53,30,16,1.88,1.17,80.0,16.9,4494,Mirage,72
1125,8789,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,5,57,44,1.30,1.19,80.7,23.7,10831,Overpass,113
460,9031,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,0,0,0,0.00,0.00,0.0,0.0,10503,Vertigo,119
310,7383,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,0,0,0,0.00,0.00,0.0,0.0,11588,Overpass,186
1331,557,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,0,0,0,0.00,0.00,0.0,0.0,9455,Ancient,254
426,9031,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,0,0,0,0.00,0.00,0.0,0.0,10503,Dust2,286
543,8727,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,0,0,0,0.00,0.00,0.0,0.0,11654,Vertigo,352
280,7168,0,0.0,0,0.0,0.0,0.0,0,0,0.0,...,20,19,13,1.46,1.11,78.9,18.8,4494,Ancient,411


In [13]:
players_feats[players_feats['p5_rounds_played'] == 0]

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
994,555,16,56.2,17,0.94,67.9,4.5,1,27,0.59,...,0,0,0,0.0,0.0,0.0,0.0,9215,Mirage,98
86,284,207,60.9,236,0.88,65.0,2.0,12,344,0.60,...,0,0,0,0.0,0.0,0.0,0.0,5995,Nuke,115
460,9031,0,0.0,0,0.00,0.0,0.0,0,0,0.00,...,0,0,0,0.0,0.0,0.0,0.0,10503,Vertigo,119
310,7383,0,0.0,0,0.00,0.0,0.0,0,0,0.00,...,0,0,0,0.0,0.0,0.0,0.0,11588,Overpass,186
1331,557,0,0.0,0,0.00,0.0,0.0,0,0,0.00,...,0,0,0,0.0,0.0,0.0,0.0,9455,Ancient,254
426,9031,0,0.0,0,0.00,0.0,0.0,0,0,0.00,...,0,0,0,0.0,0.0,0.0,0.0,10503,Dust2,286
543,8727,0,0.0,0,0.00,0.0,0.0,0,0,0.00,...,0,0,0,0.0,0.0,0.0,0.0,11654,Vertigo,352
61,7805,28,57.1,40,0.70,60.8,1.9,2,54,0.52,...,0,0,0,0.0,0.0,0.0,0.0,10399,Overpass,400
325,9221,78,52.6,41,1.90,93.7,1.7,4,87,0.90,...,0,0,0,0.0,0.0,0.0,0.0,11044,Mirage,418


#### Look at exam data whether some values are missing

In [14]:
pd.set_option('display.max_columns', None)

j = 0
for i in exam_data['map_id']:
    temp = players_feats[players_feats['map_id'] == i]
    if j == 0:
        exam_feats_df = temp
        j += 1
        
    exam_feats_df = pd.concat([exam_feats_df, temp])

It is easier for a quick glance of data in excel. Almost all elements are filled with non-zeros, except for some stats with one player.

In [15]:
exam_feats_df.to_csv('exam_feats_df.csv')

# ANN for classification (1st iteration)

For this classification an ANN will be used. I will try a few iterations of ANNs. For X values, the features of a player will be used, along with maps. Particularly:
- "map_id" will be used to merge "team1_id" with "team2_id"
- "map_name" will be hot-encoded
- train.map_id == players_feats.map_id and train.team{1|2}_id == players_feats.team_id

It is really important to correctly merge tables. Also, it is really important to apply scaler for the gradient descent to converge faster.

## Inner merge of two tables in 2 steps

In [16]:
# step 1 - merge with team 1
merged_df = pd.merge(train_data, players_feats, left_on = ['map_id', 'team1_id', 'map_name'], right_on =['map_id', 'team_id', 'map_name'])
merged_df

,map_id,team1_id,team2_id,map_name,who_win,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p1_kill_death,p1_kill_round,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds,p2_id,p2_total_kills,p2_headshots,p2_total_deaths,p2_kd_ratio,p2_damage_per_round,p2_grenade_damage_per_round,p2_maps_played,p2_rounds_played,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p2_kill_death,p2_kill_round,p2_rounds_with_kills,p2_kill_death_difference,p2_total_opening_kills,p2_total_opening_deaths,p2_opening_kill_ratio,p2_opening_kill_rating,p2_team_win_percent_after_first_kill,p2_first_kill_in_won_rounds,p3_id,p3_total_kills,p3_headshots,p3_total_deaths,p3_kd_ratio,p3_damage_per_round,p3_grenade_damage_per_round,p3_maps_played,p3_rounds_played,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p3_kill_death,p3_kill_round,p3_rounds_with_kills,p3_kill_death_difference,p3_total_opening_kills,p3_total_opening_deaths,p3_opening_kill_ratio,p3_opening_kill_rating,p3_team_win_percent_after_first_kill,p3_first_kill_in_won_rounds,p4_id,p4_total_kills,p4_headshots,p4_total_deaths,p4_kd_ratio,p4_damage_per_round,p4_grenade_damage_per_round,p4_maps_played,p4_rounds_played,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p4_kill_death,p4_kill_round,p4_rounds_with_kills,p4_kill_death_difference,p4_total_opening_kills,p4_total_opening_deaths,p4_opening_kill_ratio,p4_opening_kill_rating,p4_team_win_percent_after_first_kill,p4_first_kill_in_won_rounds,p5_id,p5_total_kills,p5_headshots,p5_total_deaths,p5_kd_ratio,p5_damage_per_round,p5_grenade_damage_per_round,p5_maps_played,p5_rounds_played,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating,p5_kill_death,p5_kill_round,p5_rounds_with_kills,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id
0,0,4869,11251,Vertigo,0,922,213,41.3,236,0.90,73.2,6.8,13,335,0.64,0.18,0.70,0.12,0.06,0.97,0.90,0.64,146,-23,34,49,0.69,0.93,67.6,12.9,11777,230,40.9,244,0.94,76.7,7.6,13,335,0.69,0.13,0.73,0.07,0.08,1.01,0.94,0.69,153,-14,48,49,0.98,1.14,72.9,19.7,16080,214,50.9,212,1.01,75.7,5.7,13,335,0.64,0.13,0.63,0.08,0.10,1.03,1.01,0.64,150,2,27,31,0.87,0.92,77.8,11.8,16848,233,35.2,212,1.10,74.2,3.2,13,335,0.70,0.13,0.63,0.09,0.08,1.07,1.10,0.70,158,21,32,18,1.78,1.05,84.4,15.2,18221,255,48.2,186,1.37,84.8,4.7,12,305,0.84,0.11,0.61,0.08,0.12,1.23,1.37,0.84,166,69,24,20,1.20,0.95,62.5,9.3,4869
1,2,4869,5752,Ancient,1,922,310,44.2,363,0.85,72.1,7.7,18,511,0.61,0.15,0.71,0.09,0.06,0.94,0.84,0.60,203,-57,55,63,0.87,1.02,72.7,16.1,2553,69,34.8,55,1.25,82.0,4.9,3,91,0.76,0.12,0.60,0.08,0.10,1.20,1.11,0.75,13,2,3,2,1.50,1.08,66.7,12.5,11777,328,50.0,350,0.94,72.6,6.0,18,511,0.64,0.14,0.68,0.10,0.09,0.99,0.93,0.64,224,-23,51,54,0.94,1.01,82.4,16.9,16080,359,51.0,329,1.09,78.2,5.9,18,511,0.70,0.17,0.64,0.07,0.08,1.09,1.09,0.70,232,27,39,42,0.93,0.93,64.1,10.0,16848,335,26.9,315,1.06,66.4,2.8,18,511,0.66,0.09,0.62,0.07,0.08,1.02,1.06,0.66,226,17,41,37,1.11,0.96,70.7,11.6,4869
2,3,10567,11595,Dust2,1,356,281,58.0,331,0.85,65.1,3.0,20,510,0.55,0.13,0.65,0.08,0.07,0.91,0.83,0.54,142,-40,31,54,0.57,0.85,67.7,11.6,630,321,50.8,335,0.96,74.1,8.2,20,510,0.63,0.16,0.66,0.09,0.10,1.03,0.95,0.62,156,-13,37,32,1.16,1.03,62.2,12.7,2241,379

In [17]:
# step 2 - merge with team 2
train_merged_df = pd.merge(merged_df, players_feats, left_on = ['map_id', 'team2_id', 'map_name'], right_on = ['map_id', 'team_id', 'map_name'], suffixes = ('_team_1', '_team_2'))
train_merged_df

,map_id,team1_id,team2_id,map_name,who_win,p1_id_team_1,p1_total_kills_team_1,p1_headshots_team_1,p1_total_deaths_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_grenade_damage_per_round_team_1,p1_maps_played_team_1,p1_rounds_played_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p1_kill_death_team_1,p1_kill_round_team_1,p1_rounds_with_kills_team_1,p1_kill_death_difference_team_1,p1_total_opening_kills_team_1,p1_total_opening_deaths_team_1,p1_opening_kill_ratio_team_1,p1_opening_kill_rating_team_1,p1_team_win_percent_after_first_kill_team_1,p1_first_kill_in_won_rounds_team_1,p2_id_team_1,p2_total_kills_team_1,p2_headshots_team_1,p2_total_deaths_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_grenade_damage_per_round_team_1,p2_maps_played_team_1,p2_rounds_played_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p2_kill_death_team_1,p2_kill_round_team_1,p2_rounds_with_kills_team_1,p2_kill_death_difference_team_1,p2_total_opening_kills_team_1,p2_total_opening_deaths_team_1,p2_opening_kill_ratio_team_1,p2_opening_kill_rating_team_1,p2_team_win_percent_after_first_kill_team_1,p2_first_kill_in_won_rounds_team_1,p3_id_team_1,p3_total_kills_team_1,p3_headshots_team_1,p3_total_deaths_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_grenade_damage_per_round_team_1,p3_maps_played_team_1,p3_rounds_played_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p3_kill_death_team_1,p3_kill_round_team_1,p3_rounds_with_kills_team_1,p3_kill_death_difference_team_1,p3_total_opening_kills_team_1,p3_total_opening_deaths_team_1,p3_opening_kill_ratio_team_1,p3_opening_kill_rating_team_1,p3_team_win_percent_after_first_kill_team_1,p3_first_kill_in_won_rounds_team_1,p4_id_team_1,p4_total_kills_team_1,p4_headshots_team_1,p4_total_deaths_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_grenade_damage_per_round_team_1,p4_maps_played_team_1,p4_rounds_played_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p4_kill_death_team_1,p4_kill_round_team_1,p4_rounds_with_kills_team_1,p4_kill_death_difference_team_1,p4_total_opening_kills_team_1,p4_total_opening_deaths_team_1,p4_opening_kill_ratio_team_1,p4_opening_kill_rating_team_1,p4_team_win_percent_after_first_kill_team_1,p4_first_kill_in_won_rounds_team_1,p5_id_team_1,p5_total_kills_team_1,p5_headshots_team_1,p5_total_deaths_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_grenade_damage_per_round_team_1,p5_maps_played_team_1,p5_rounds_played_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p5_kill_death_team_1,p5_kill_round_team_1,p5_rounds_with_kills_team_1,p5_kill_death_difference_team_1,p5_total_opening_kills_team_1,p5_total_opening_deaths_team_1,p5_opening_kill_ratio_team_1,p5_opening_kill_rating_team_1,p5_team_win_percent_after_first_kill_team_1,p5_first_kill_in_won_rounds_team_1,team_id_team_1,p1_id_team_2,p1_total_kills_team_2,p1_headshots_team_2,p1_total_deaths_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_grenade_damage_per_round_team_2,p1_maps_played_team_2,p1_rounds_played_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p1_kill_death_team_2,p1_kill_round_team_2,p1_rounds_with_kills_team_2,p1_kill_death_difference_team_2,p1_total_opening_kills_team_2,p1_total_opening_deaths_team_2,p1_opening_k

## Prepare X and y for ANN

In [18]:
X = train_merged_df.drop(columns=['map_id','who_win', 'team1_id', 'team2_id'])
X = pd.get_dummies(X, columns=['map_name'])
X

,p1_id_team_1,p1_total_kills_team_1,p1_headshots_team_1,p1_total_deaths_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_grenade_damage_per_round_team_1,p1_maps_played_team_1,p1_rounds_played_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p1_kill_death_team_1,p1_kill_round_team_1,p1_rounds_with_kills_team_1,p1_kill_death_difference_team_1,p1_total_opening_kills_team_1,p1_total_opening_deaths_team_1,p1_opening_kill_ratio_team_1,p1_opening_kill_rating_team_1,p1_team_win_percent_after_first_kill_team_1,p1_first_kill_in_won_rounds_team_1,p2_id_team_1,p2_total_kills_team_1,p2_headshots_team_1,p2_total_deaths_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_grenade_damage_per_round_team_1,p2_maps_played_team_1,p2_rounds_played_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p2_kill_death_team_1,p2_kill_round_team_1,p2_rounds_with_kills_team_1,p2_kill_death_difference_team_1,p2_total_opening_kills_team_1,p2_total_opening_deaths_team_1,p2_opening_kill_ratio_team_1,p2_opening_kill_rating_team_1,p2_team_win_percent_after_first_kill_team_1,p2_first_kill_in_won_rounds_team_1,p3_id_team_1,p3_total_kills_team_1,p3_headshots_team_1,p3_total_deaths_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_grenade_damage_per_round_team_1,p3_maps_played_team_1,p3_rounds_played_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p3_kill_death_team_1,p3_kill_round_team_1,p3_rounds_with_kills_team_1,p3_kill_death_difference_team_1,p3_total_opening_kills_team_1,p3_total_opening_deaths_team_1,p3_opening_kill_ratio_team_1,p3_opening_kill_rating_team_1,p3_team_win_percent_after_first_kill_team_1,p3_first_kill_in_won_rounds_team_1,p4_id_team_1,p4_total_kills_team_1,p4_headshots_team_1,p4_total_deaths_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_grenade_damage_per_round_team_1,p4_maps_played_team_1,p4_rounds_played_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p4_kill_death_team_1,p4_kill_round_team_1,p4_rounds_with_kills_team_1,p4_kill_death_difference_team_1,p4_total_opening_kills_team_1,p4_total_opening_deaths_team_1,p4_opening_kill_ratio_team_1,p4_opening_kill_rating_team_1,p4_team_win_percent_after_first_kill_team_1,p4_first_kill_in_won_rounds_team_1,p5_id_team_1,p5_total_kills_team_1,p5_headshots_team_1,p5_total_deaths_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_grenade_damage_per_round_team_1,p5_maps_played_team_1,p5_rounds_played_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p5_kill_death_team_1,p5_kill_round_team_1,p5_rounds_with_kills_team_1,p5_kill_death_difference_team_1,p5_total_opening_kills_team_1,p5_total_opening_deaths_team_1,p5_opening_kill_ratio_team_1,p5_opening_kill_rating_team_1,p5_team_win_percent_after_first_kill_team_1,p5_first_kill_in_won_rounds_team_1,team_id_team_1,p1_id_team_2,p1_total_kills_team_2,p1_headshots_team_2,p1_total_deaths_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_grenade_damage_per_round_team_2,p1_maps_played_team_2,p1_rounds_played_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p1_kill_death_team_2,p1_kill_round_team_2,p1_rounds_with_kills_team_2,p1_kill_death_difference_team_2,p1_total_opening_kills_team_2,p1_total_opening_deaths_team_2,p1_opening_kill_ratio_team_2,p1_opening_kill_rating_te

In [19]:
y = train_merged_df['who_win']
y

0      0
1      1
2      1
3      1
4      1
      ..
692    1
693    1
694    0
695    1
696    0
Name: who_win, Length: 697, dtype: int64

## Apply MinMaxScaler

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X.values), columns=X.columns, index=X.index)
X

,p1_id_team_1,p1_total_kills_team_1,p1_headshots_team_1,p1_total_deaths_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_grenade_damage_per_round_team_1,p1_maps_played_team_1,p1_rounds_played_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p1_kill_death_team_1,p1_kill_round_team_1,p1_rounds_with_kills_team_1,p1_kill_death_difference_team_1,p1_total_opening_kills_team_1,p1_total_opening_deaths_team_1,p1_opening_kill_ratio_team_1,p1_opening_kill_rating_team_1,p1_team_win_percent_after_first_kill_team_1,p1_first_kill_in_won_rounds_team_1,p2_id_team_1,p2_total_kills_team_1,p2_headshots_team_1,p2_total_deaths_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_grenade_damage_per_round_team_1,p2_maps_played_team_1,p2_rounds_played_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p2_kill_death_team_1,p2_kill_round_team_1,p2_rounds_with_kills_team_1,p2_kill_death_difference_team_1,p2_total_opening_kills_team_1,p2_total_opening_deaths_team_1,p2_opening_kill_ratio_team_1,p2_opening_kill_rating_team_1,p2_team_win_percent_after_first_kill_team_1,p2_first_kill_in_won_rounds_team_1,p3_id_team_1,p3_total_kills_team_1,p3_headshots_team_1,p3_total_deaths_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_grenade_damage_per_round_team_1,p3_maps_played_team_1,p3_rounds_played_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p3_kill_death_team_1,p3_kill_round_team_1,p3_rounds_with_kills_team_1,p3_kill_death_difference_team_1,p3_total_opening_kills_team_1,p3_total_opening_deaths_team_1,p3_opening_kill_ratio_team_1,p3_opening_kill_rating_team_1,p3_team_win_percent_after_first_kill_team_1,p3_first_kill_in_won_rounds_team_1,p4_id_team_1,p4_total_kills_team_1,p4_headshots_team_1,p4_total_deaths_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_grenade_damage_per_round_team_1,p4_maps_played_team_1,p4_rounds_played_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p4_kill_death_team_1,p4_kill_round_team_1,p4_rounds_with_kills_team_1,p4_kill_death_difference_team_1,p4_total_opening_kills_team_1,p4_total_opening_deaths_team_1,p4_opening_kill_ratio_team_1,p4_opening_kill_rating_team_1,p4_team_win_percent_after_first_kill_team_1,p4_first_kill_in_won_rounds_team_1,p5_id_team_1,p5_total_kills_team_1,p5_headshots_team_1,p5_total_deaths_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_grenade_damage_per_round_team_1,p5_maps_played_team_1,p5_rounds_played_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p5_kill_death_team_1,p5_kill_round_team_1,p5_rounds_with_kills_team_1,p5_kill_death_difference_team_1,p5_total_opening_kills_team_1,p5_total_opening_deaths_team_1,p5_opening_kill_ratio_team_1,p5_opening_kill_rating_team_1,p5_team_win_percent_after_first_kill_team_1,p5_first_kill_in_won_rounds_team_1,team_id_team_1,p1_id_team_2,p1_total_kills_team_2,p1_headshots_team_2,p1_total_deaths_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_grenade_damage_per_round_team_2,p1_maps_played_team_2,p1_rounds_played_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p1_kill_death_team_2,p1_kill_round_team_2,p1_rounds_with_kills_team_2,p1_kill_death_difference_team_2,p1_total_opening_kills_team_2,p1_total_opening_deaths_team_2,p1_opening_kill_ratio_team_2,p1_opening_kill_rating_te

In [21]:
X.drop(labels=0, axis=0)

,p1_id_team_1,p1_total_kills_team_1,p1_headshots_team_1,p1_total_deaths_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_grenade_damage_per_round_team_1,p1_maps_played_team_1,p1_rounds_played_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p1_kill_death_team_1,p1_kill_round_team_1,p1_rounds_with_kills_team_1,p1_kill_death_difference_team_1,p1_total_opening_kills_team_1,p1_total_opening_deaths_team_1,p1_opening_kill_ratio_team_1,p1_opening_kill_rating_team_1,p1_team_win_percent_after_first_kill_team_1,p1_first_kill_in_won_rounds_team_1,p2_id_team_1,p2_total_kills_team_1,p2_headshots_team_1,p2_total_deaths_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_grenade_damage_per_round_team_1,p2_maps_played_team_1,p2_rounds_played_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p2_kill_death_team_1,p2_kill_round_team_1,p2_rounds_with_kills_team_1,p2_kill_death_difference_team_1,p2_total_opening_kills_team_1,p2_total_opening_deaths_team_1,p2_opening_kill_ratio_team_1,p2_opening_kill_rating_team_1,p2_team_win_percent_after_first_kill_team_1,p2_first_kill_in_won_rounds_team_1,p3_id_team_1,p3_total_kills_team_1,p3_headshots_team_1,p3_total_deaths_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_grenade_damage_per_round_team_1,p3_maps_played_team_1,p3_rounds_played_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p3_kill_death_team_1,p3_kill_round_team_1,p3_rounds_with_kills_team_1,p3_kill_death_difference_team_1,p3_total_opening_kills_team_1,p3_total_opening_deaths_team_1,p3_opening_kill_ratio_team_1,p3_opening_kill_rating_team_1,p3_team_win_percent_after_first_kill_team_1,p3_first_kill_in_won_rounds_team_1,p4_id_team_1,p4_total_kills_team_1,p4_headshots_team_1,p4_total_deaths_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_grenade_damage_per_round_team_1,p4_maps_played_team_1,p4_rounds_played_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p4_kill_death_team_1,p4_kill_round_team_1,p4_rounds_with_kills_team_1,p4_kill_death_difference_team_1,p4_total_opening_kills_team_1,p4_total_opening_deaths_team_1,p4_opening_kill_ratio_team_1,p4_opening_kill_rating_team_1,p4_team_win_percent_after_first_kill_team_1,p4_first_kill_in_won_rounds_team_1,p5_id_team_1,p5_total_kills_team_1,p5_headshots_team_1,p5_total_deaths_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_grenade_damage_per_round_team_1,p5_maps_played_team_1,p5_rounds_played_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p5_kill_death_team_1,p5_kill_round_team_1,p5_rounds_with_kills_team_1,p5_kill_death_difference_team_1,p5_total_opening_kills_team_1,p5_total_opening_deaths_team_1,p5_opening_kill_ratio_team_1,p5_opening_kill_rating_team_1,p5_team_win_percent_after_first_kill_team_1,p5_first_kill_in_won_rounds_team_1,team_id_team_1,p1_id_team_2,p1_total_kills_team_2,p1_headshots_team_2,p1_total_deaths_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_grenade_damage_per_round_team_2,p1_maps_played_team_2,p1_rounds_played_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p1_kill_death_team_2,p1_kill_round_team_2,p1_rounds_with_kills_team_2,p1_kill_death_difference_team_2,p1_total_opening_kills_team_2,p1_total_opening_deaths_team_2,p1_opening_kill_ratio_team_2,p1_opening_kill_rating_te

## Train ANN

In [22]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras import regularizers

In [23]:
X.shape

(697, 259)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30/697)

In [25]:
X_train.shape

(666, 259)

In [26]:
X_test.shape

(31, 259)

In [27]:
no_inputs_X = X_train.shape[1]

In [28]:
model_ANN = keras.Sequential([
    keras.layers.Dense(3, activation='tanh', input_shape=(no_inputs_X,), kernel_regularizer=regularizers.l2(0.0001)),
    keras.layers.Dense(2, activation='tanh', kernel_regularizer=regularizers.l2(0.0001)),
    keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001))
])

model_ANN.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [60]:
%%capture 
model_ANN.fit(X_train, y_train, epochs=600);

In [61]:
test_loss, test_accuracy = model_ANN.evaluate(X_train, y_train)

21/21 [==============================] - 0s 550us/step - loss: 0.3569 - accuracy: 0.8348


In [62]:
test_loss, test_accuracy = model_ANN.evaluate(X_test, y_test)

1/1 [==============================] - 0s 15ms/step - loss: 1.3207 - accuracy: 0.4839


I have not used an ANN with more layers as we are facing the overfitting problem. Here are some sample results:
0.6452
0.6129
0.4194
0.5806
0.5161
0.4516
0.5484
0.3871
0.5161
0.6452
0.5806

In [32]:
sample_results = [0.6452, 0.6129,0.4194, 0.5806, 0.5161, 0.4516, 0.5484, 0.3871, 0.5161, 0.6452, 0.5806]
print(np.mean(sample_results),np.std(sample_results))

0.5366545454545455 0.0839764075487478


## Classis MLs with crossval

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

models = [KNeighborsClassifier(n_neighbors=13), LinearDiscriminantAnalysis(), GaussianNB(), LogisticRegression(solver='liblinear', max_iter=1000)]
for model in models:
    scores = cross_val_score(model, X, y, cv=20)
    print(f'{model} mean score {round(scores.mean(),3)}, score std {round(scores.std(),4)}')
    print(scores)


KNeighborsClassifier(n_neighbors=13) mean score 0.561, score std 0.0814
[0.6        0.57142857 0.42857143 0.51428571 0.6        0.51428571
 0.48571429 0.62857143 0.54285714 0.62857143 0.57142857 0.57142857
 0.54285714 0.6        0.45714286 0.68571429 0.71428571 0.38235294
 0.52941176 0.64705882]
LinearDiscriminantAnalysis() mean score 0.564, score std 0.0932
[0.65714286 0.54285714 0.51428571 0.54285714 0.54285714 0.48571429
 0.74285714 0.34285714 0.57142857 0.51428571 0.62857143 0.68571429
 0.4        0.6        0.54285714 0.54285714 0.62857143 0.67647059
 0.5        0.61764706]
GaussianNB() mean score 0.526, score std 0.0656
[0.6        0.45714286 0.57142857 0.37142857 0.51428571 0.54285714
 0.48571429 0.57142857 0.57142857 0.57142857 0.57142857 0.51428571
 0.68571429 0.54285714 0.45714286 0.48571429 0.51428571 0.47058824
 0.47058824 0.55882353]
LogisticRegression(max_iter=1000, solver='liblinear') mean score 0.613, score std 0.0934
[0.65714286 0.54285714 0.68571429 0.6        0.48571

# ANN for classification (2nd iteration)

The previous ANN shows that ANN overfits. I will reduce number of columns. Many columns can be calculated from other columns. There are 25 features per player, which includes hid id number. At least, 8 features can be calculated from other columns. Those are:
- kd_ratio
- kills_per_round
- deaths_per_round
- rating (ambigious feature, might be KAST)
- kill_round
- kill_death_difference
- opening_kill_ratio
- opening_kill_rating

Also, some players have absent values or half-absent values. A player's performance will vary according to a map and a team they play against. As someone, who know this game and also an active player, I believe that these statistics are important:
- headshots
- kd_ratio
- damage per round
- kills per round
- assists per round
- deaths per round
- saved by teammate per round
- saved teammates per round

## Feature engineering

In [34]:
# preserve only important columns
import_col = [
    'map_id','map_name','team_id',
    'p1_kills_per_round','p1_kd_ratio','p1_damage_per_round', 'p1_kills_per_round','p1_assists_per_round', 'p1_deaths_per_round', 'p1_saved_by_teammate_per_round', 'p1_saved_teammates_per_round','p1_rating',
    'p2_kills_per_round','p2_kd_ratio','p2_damage_per_round', 'p2_kills_per_round','p2_assists_per_round', 'p2_deaths_per_round', 'p2_saved_by_teammate_per_round', 'p2_saved_teammates_per_round','p2_rating',
    'p3_kills_per_round','p3_kd_ratio','p3_damage_per_round', 'p3_kills_per_round','p3_assists_per_round', 'p3_deaths_per_round', 'p3_saved_by_teammate_per_round', 'p3_saved_teammates_per_round','p3_rating',
    'p4_kills_per_round','p4_kd_ratio','p4_damage_per_round', 'p4_kills_per_round','p4_assists_per_round', 'p4_deaths_per_round', 'p4_saved_by_teammate_per_round', 'p4_saved_teammates_per_round','p4_rating',
    'p5_kills_per_round','p5_kd_ratio','p5_damage_per_round', 'p5_kills_per_round','p5_assists_per_round', 'p5_deaths_per_round', 'p5_saved_by_teammate_per_round', 'p5_saved_teammates_per_round','p5_rating'
             ]

players_feats_2 = players_feats[import_col]
players_feats_2

,map_id,map_name,team_id,p1_kills_per_round,p1_kd_ratio,p1_damage_per_round,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p2_kills_per_round,p2_kd_ratio,p2_damage_per_round,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p3_kills_per_round,p3_kd_ratio,p3_damage_per_round,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p4_kills_per_round,p4_kd_ratio,p4_damage_per_round,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p5_kills_per_round,p5_kd_ratio,p5_damage_per_round,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating
1152,0,Vertigo,4869,0.64,0.90,73.2,0.64,0.18,0.70,0.12,0.06,0.97,0.69,0.94,76.7,0.69,0.13,0.73,0.07,0.08,1.01,0.64,1.01,75.7,0.64,0.13,0.63,0.08,0.10,1.03,0.70,1.10,74.2,0.70,0.13,0.63,0.09,0.08,1.07,0.84,1.37,84.8,0.84,0.11,0.61,0.08,0.12,1.23
1153,0,Vertigo,11251,0.81,1.24,93.1,0.81,0.20,0.66,0.11,0.15,1.34,0.73,1.34,73.9,0.73,0.11,0.55,0.07,0.10,1.16,0.70,1.11,73.8,0.70,0.12,0.63,0.13,0.08,1.10,0.72,1.03,79.6,0.72,0.14,0.70,0.14,0.10,1.13,0.63,1.08,71.0,0.63,0.14,0.58,0.09,0.09,1.06
1424,1,Nuke,8297,0.64,0.81,78.9,0.64,0.16,0.79,0.09,0.07,0.96,0.75,1.19,78.0,0.75,0.10,0.63,0.07,0.09,1.11,0.71,1.11,79.3,0.71,0.13,0.64,0.07,0.11,1.10,0.57,0.84,58.4,0.57,0.10,0.68,0.08,0.07,0.86,0.56,0.87,63.3,0.56,0.12,0.64,0.11,0.07,0.94
1425,1,Nuke,7020,0.69,1.11,70.6,0.69,0.09,0.62,0.12,0.10,1.05,0.60,0.97,69.1,0.60,0.11,0.62,0.10,0.08,1.01,0.69,1.08,73.0,0.69,0.12,0.64,0.13,0.11,1.08,0.74,1.07,84.2,0.74,0.16,0.69,0.06,0.12,1.15,0.80,1.48,85.0,0.80,0.11,0.54,0.04,0.15,1.31
1059,2,Ancient,5752,0.76,1.17,86.0,0.76,0.16,0.65,0.10,0.05,1.21,0.59,0.94,64.8,0.59,0.15,0.62,0.11,0.08,0.96,0.75,1.22,80.1,0.75,0.11,0.61,0.05,0.08,1.17,0.88,1.79,82.9,0.88,0.10,0.49,0.04,0.13,1.41,0.56,0.83,68.8,0.56,0.17,0.67,0.11,0.07,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,740,Nuke,4494,0.60,0.89,64.3,0.60,0.11,0.68,0.10,0.07,0.92,0.73,1.23,78.9,0.73,0.11,0.59,0.06,0.10,1.17,0.60,0.88,71.3,0.60,0.13,0.68,0.10,0.08,0.96,0.73,1.23,72.4,0.73,0.08,0.59,0.07,0.10,1.14,0.72,1.15,81.4,0.72,0.14,0.63,0.07,0.08,1.17
30,741,Overpass,9565,0.64,0.99,80.5,0.64,0.21,0.65,0.07,0.09,1.07,0.62,0.88,69.6,0.62,0.19,0.70,0.08,0.11,0.96,0.59,0.92,71.4,0.59,0.07,0.64,0.05,0.08,0.94,0.78,1.28,80.6,0.78,0.09,0.61,0.07,0.13,1.23,0.69,1.03,73.5,0.69,0.09,0.67,0.12,0.13,1.07
31,741,Overpass,6667,0.74,1.08,78.5,0.74,0.15,0.68,0.09,0.08,1.10,0.72,1.04,84.5,0.72,0.17,0.70,0.07,0.07,1.12,0.75,1.29,77.2,0.75,0.11,0.59,0.09,0.14,1.18,0.83,1.23,87.9,0.83,0.11,0.68,0.08,0.08,1.19,0.78,1.35,80.7,0.78,0.11,0.58,0.14,0.10,1.22
10,742,Inferno,6667,0.67,0.95,79.9,0.67,0.15,0.70,0.13,0.06,1.09,0.66,1.00,73.9,0.66,0.14,0.65,0.07,0.10,1.07,0.61,1.01,68.1,0.61,0.12,0.60,0.10,0.09,1.05,0.70,1.23,76.6,0.70,0.12,0.57,0.07,0.05,1.15,0.69,1.21,70.1,0.69,0.10,0.57,0.08,0.11,1.10


## Inner merge of two tables in 2 steps

In [35]:
# step 1 - merge with team 1
merged_df = pd.merge(train_data, players_feats_2, left_on = ['map_id', 'team1_id', 'map_name'], right_on =['map_id', 'team_id', 'map_name'])
merged_df

,map_id,team1_id,team2_id,map_name,who_win,team_id,p1_kills_per_round,p1_kd_ratio,p1_damage_per_round,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p2_kills_per_round,p2_kd_ratio,p2_damage_per_round,p2_kills_per_round,p2_assists_per_round,p2_deaths_per_round,p2_saved_by_teammate_per_round,p2_saved_teammates_per_round,p2_rating,p3_kills_per_round,p3_kd_ratio,p3_damage_per_round,p3_kills_per_round,p3_assists_per_round,p3_deaths_per_round,p3_saved_by_teammate_per_round,p3_saved_teammates_per_round,p3_rating,p4_kills_per_round,p4_kd_ratio,p4_damage_per_round,p4_kills_per_round,p4_assists_per_round,p4_deaths_per_round,p4_saved_by_teammate_per_round,p4_saved_teammates_per_round,p4_rating,p5_kills_per_round,p5_kd_ratio,p5_damage_per_round,p5_kills_per_round,p5_assists_per_round,p5_deaths_per_round,p5_saved_by_teammate_per_round,p5_saved_teammates_per_round,p5_rating
0,0,4869,11251,Vertigo,0,4869,0.64,0.90,73.2,0.64,0.18,0.70,0.12,0.06,0.97,0.69,0.94,76.7,0.69,0.13,0.73,0.07,0.08,1.01,0.64,1.01,75.7,0.64,0.13,0.63,0.08,0.10,1.03,0.70,1.10,74.2,0.70,0.13,0.63,0.09,0.08,1.07,0.84,1.37,84.8,0.84,0.11,0.61,0.08,0.12,1.23
1,2,4869,5752,Ancient,1,4869,0.61,0.85,72.1,0.61,0.15,0.71,0.09,0.06,0.94,0.76,1.25,82.0,0.76,0.12,0.60,0.08,0.10,1.20,0.64,0.94,72.6,0.64,0.14,0.68,0.10,0.09,0.99,0.70,1.09,78.2,0.70,0.17,0.64,0.07,0.08,1.09,0.66,1.06,66.4,0.66,0.09,0.62,0.07,0.08,1.02
2,3,10567,11595,Dust2,1,10567,0.55,0.85,65.1,0.55,0.13,0.65,0.08,0.07,0.91,0.63,0.96,74.1,0.63,0.16,0.66,0.09,0.10,1.03,0.74,1.09,80.9,0.74,0.13,0.68,0.11,0.11,1.12,0.65,0.99,69.5,0.65,0.09,0.65,0.08,0.10,0.96,0.72,1.20,74.3,0.72,0.11,0.60,0.09,0.07,1.17
3,4,8297,4773,Vertigo,1,8297,0.78,1.20,87.1,0.78,0.18,0.65,0.17,0.04,1.36,0.79,1.52,96.5,0.79,0.31,0.52,0.09,0.14,1.42,0.86,1.89,89.4,0.86,0.09,0.46,0.10,0.11,1.46,0.98,1.68,97.0,0.98,0.14,0.58,0.12,0.15,1.50,0.68,1.39,65.4,0.68,0.14,0.49,0.13,0.10,1.19
4,5,7020,4869,Mirage,1,7020,0.71,1.04,76.2,0.71,0.10,0.68,0.06,0.08,1.02,0.71,1.13,75.9,0.71,0.06,0.63,0.09,0.06,1.13,0.65,0.98,70.0,0.65,0.10,0.66,0.10,0.11,0.97,0.56,0.83,64.7,0.56,0.11,0.68,0.10,0.09,0.90,0.68,0.89,71.9,0.68,0.09,0.76,0.07,0.10,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,738,4869,6667,Mirage,1,4869,0.55,0.84,66.2,0.55,0.14,0.65,0.06,0.04,0.92,0.60,0.83,65.9,0.60,0.09,0.73,0.07,0.06,0.91,0.69,1.00,78.1,0.69,0.12,0.69,0.06,0.07,1.03,0.79,1.22,86.3,0.79,0.14,0.65,0.08,0.08,1.23,0.72,1.16,70.8,0.72,0.07,0.62,0.08,0.08,1.08
704,739,5973,7020,Ancient,1,5973,0.61,0.92,69.7,0.61,0.12,0.67,0.09,0.10,0.97,0.74,1.10,83.4,0.74,0.15,0.67,0.09,0.12,1.12,0.78,1.05,86.5,0.78,0.15,0.74,0.08,0.09,1.15,0.63,0.97,67.2,0.63,0.10,0.64,0.10,0.10,1.00,0.86,1.22,92.3,0.86,0.14,0.70,0.08,0.10,1.22
705,740,4869,4494,Nuke,0,4869,0.66,1.09,71.5,0.66,0.09,0.61,0.08,0.07,1.06,0.66,1.00,75.1,0.66,0.14,0.67,0.10,0.10,1.04,0.71,1.07,80.6,0.71,0.15,0.66,0.09,0.08,1.13,0.74,1.30,75.6,0.74,0.11,0.57,0.08,0.06,1.19,0.77,1.21,79.9,0.77,0.11,0.64,0.08,0.12,1.17
706,741,9565,6667,Overpass,1,9565,0.64,0.99,80.5,0.64,0.21,0.65,0.07,0.09,1.07,0.62,0.88,69.6,0.62,0.19,0.70,0.08,0.11,0.96,0.59,0.92,71.4,0.59,0.07,0.64,0.05,0.08,0.94,0.78,1.28,80.6,0.78,0.09,0.61,0.07,0.13,1.23,0.69,1.03,73.5,0.69,0.09,0.67,0.12,0.13,1.07


In [36]:
# step 2 - merge with team 2
train_merged_df = pd.merge(merged_df, players_feats_2, left_on = ['map_id', 'team2_id', 'map_name'], right_on = ['map_id', 'team_id', 'map_name'], suffixes = ('_team_1', '_team_2'))
train_merged_df

,map_id,team1_id,team2_id,map_name,who_win,team_id_team_1,p1_kills_per_round_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p2_kills_per_round_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p3_kills_per_round_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p4_kills_per_round_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p5_kills_per_round_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,team_id_team_2,p1_kills_per_round_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p2_kills_per_round_team_2,p2_kd_ratio_team_2,p2_damage_per_round_team_2,p2_kills_per_round_team_2,p2_assists_per_round_team_2,p2_deaths_per_round_team_2,p2_saved_by_teammate_per_round_team_2,p2_saved_teammates_per_round_team_2,p2_rating_team_2,p3_kills_per_round_team_2,p3_kd_ratio_team_2,p3_damage_per_round_team_2,p3_kills_per_round_team_2,p3_assists_per_round_team_2,p3_deaths_per_round_team_2,p3_saved_by_teammate_per_round_team_2,p3_saved_teammates_per_round_team_2,p3_rating_team_2,p4_kills_per_round_team_2,p4_kd_ratio_team_2,p4_damage_per_round_team_2,p4_kills_per_round_team_2,p4_assists_per_round_team_2,p4_deaths_per_round_team_2,p4_saved_by_teammate_per_round_team_2,p4_saved_teammates_per_round_team_2,p4_rating_team_2,p5_kills_per_round_team_2,p5_kd_ratio_team_2,p5_damage_per_round_team_2,p5_kills_per_round_team_2,p5_assists_per_round_team_2,p5_deaths_per_round_team_2,p5_saved_by_teammate_per_round_team_2,p5_saved_teammates_per_round_team_2,p5_rating_team_2
0,0,4869,11251,Vertigo,0,4869,0.64,0.90,73.2,0.64,0.18,0.70,0.12,0.06,0.97,0.69,0.94,76.7,0.69,0.13,0.73,0.07,0.08,1.01,0.64,1.01,75.7,0.64,0.13,0.63,0.08,0.10,1.03,0.70,1.10,74.2,0.70,0.13,0.63,0.09,0.08,1.07,0.84,1.37,84.8,0.84,0.11,0.61,0.08,0.12,1.23,11251,0.81,1.24,93.1,0.81,0.20,0.66,0.11,0.15,1.34,0.73,1.34,73.9,0.73,0.11,0.55,0.07,0.10,1.16,0.70,1.11,73.8,0.70,0.12,0.63,0.13,0.08,1.10,0.72,1.03,79.6,0.72,0.14,0.70,0.14,0.10,1.13,0.63,1.08,71.0,0.63,0.14,0.58,0.09,0.09,1.06
1,2,4869,5752,Ancient,1,4869,0.61,0.85,72.1,0.61,0.15,0.71,0.09,0.06,0.94,0.76,1.25,82.0,0.76,0.12,0.60,0.08,0.10,1.20,0.64,0.94,72.6,0.64,0.14,0.68,0.10,0.09,0.99,0.70,1.09,78.2,0.70,0.17,0.64,0.07,0.08,1.09,0.66,1.06,66.4,0.66,0.09,0.62,0.07,0.08,1.02,5752,0.76,1.17,86.0,0.76,0.16,0.65,0.10,0.05,1.21,0.59,0.94,64.8,0.59,0.15,0.62,0.11,0.08,0.96,0.75,1.22,80.1,0.75,0.11,0.61,0.05,0.08,1.17,0.88,1.79,82.9,0.88,0.10,0.49,0.04,0.13,1.41,0.56,0.83,68.8,0.56,0.17,0.67,0.11,0.07,0.95
2,3,10567,11595,Dust2,1,10567,0.55,0.85,65.1,0.55,0.13,0.65,0.08,0.07,0.91,0.63,0.96,74.1,0.63,0.16,0.66,0.09,0.10,1.03,0.74,1.09,80.9,0.74,0.13,0.68,0.11,0.11,1.12,0.65,0.99,69.5,0.65,0.09,0.65,0.08,0.10,0.96,0.72,1.20,74.3,0.72,0.11,0.60,0.09,0.07,1.17,11595,0.54,0.91,58.9,0.54,0.10,0.59,0.05,0.06,0.90,0.67,1.09,75.3,0.67,0.16,0.62,0.11,0.07,1.10,0.65,1.14,73.1,0.65,0.09,0.57,0.10,0.06,1.11,0.71,1.56,75.6,0.71,0.10,0.46,0.04,0.08,1.26,0.80,1.25,86.2,0.80,0.11,0.64,0.07,0.09,1.24
3,4,8297,4773,Vertigo,1,8297,0.

## Prepare X and y for ANN

In [37]:
X = train_merged_df.drop(columns=['map_id','who_win', 'team1_id', 'team2_id', 'map_name', 'team_id_team_1', 'team_id_team_2'])
X

,p1_kills_per_round_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p2_kills_per_round_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p3_kills_per_round_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p4_kills_per_round_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p5_kills_per_round_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p1_kills_per_round_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p2_kills_per_round_team_2,p2_kd_ratio_team_2,p2_damage_per_round_team_2,p2_kills_per_round_team_2,p2_assists_per_round_team_2,p2_deaths_per_round_team_2,p2_saved_by_teammate_per_round_team_2,p2_saved_teammates_per_round_team_2,p2_rating_team_2,p3_kills_per_round_team_2,p3_kd_ratio_team_2,p3_damage_per_round_team_2,p3_kills_per_round_team_2,p3_assists_per_round_team_2,p3_deaths_per_round_team_2,p3_saved_by_teammate_per_round_team_2,p3_saved_teammates_per_round_team_2,p3_rating_team_2,p4_kills_per_round_team_2,p4_kd_ratio_team_2,p4_damage_per_round_team_2,p4_kills_per_round_team_2,p4_assists_per_round_team_2,p4_deaths_per_round_team_2,p4_saved_by_teammate_per_round_team_2,p4_saved_teammates_per_round_team_2,p4_rating_team_2,p5_kills_per_round_team_2,p5_kd_ratio_team_2,p5_damage_per_round_team_2,p5_kills_per_round_team_2,p5_assists_per_round_team_2,p5_deaths_per_round_team_2,p5_saved_by_teammate_per_round_team_2,p5_saved_teammates_per_round_team_2,p5_rating_team_2
0,0.64,0.90,73.2,0.64,0.18,0.70,0.12,0.06,0.97,0.69,0.94,76.7,0.69,0.13,0.73,0.07,0.08,1.01,0.64,1.01,75.7,0.64,0.13,0.63,0.08,0.10,1.03,0.70,1.10,74.2,0.70,0.13,0.63,0.09,0.08,1.07,0.84,1.37,84.8,0.84,0.11,0.61,0.08,0.12,1.23,0.81,1.24,93.1,0.81,0.20,0.66,0.11,0.15,1.34,0.73,1.34,73.9,0.73,0.11,0.55,0.07,0.10,1.16,0.70,1.11,73.8,0.70,0.12,0.63,0.13,0.08,1.10,0.72,1.03,79.6,0.72,0.14,0.70,0.14,0.10,1.13,0.63,1.08,71.0,0.63,0.14,0.58,0.09,0.09,1.06
1,0.61,0.85,72.1,0.61,0.15,0.71,0.09,0.06,0.94,0.76,1.25,82.0,0.76,0.12,0.60,0.08,0.10,1.20,0.64,0.94,72.6,0.64,0.14,0.68,0.10,0.09,0.99,0.70,1.09,78.2,0.70,0.17,0.64,0.07,0.08,1.09,0.66,1.06,66.4,0.66,0.09,0.62,0.07,0.08,1.02,0.76,1.17,86.0,0.76,0.16,0.65,0.10,0.05,1.21,0.59,0.94,64.8,0.59,0.15,0.62,0.11,0.08,0.96,0.75,1.22,80.1,0.75,0.11,0.61,0.05,0.08,1.17,0.88,1.79,82.9,0.88,0.10,0.49,0.04,0.13,1.41,0.56,0.83,68.8,0.56,0.17,0.67,0.11,0.07,0.95
2,0.55,0.85,65.1,0.55,0.13,0.65,0.08,0.07,0.91,0.63,0.96,74.1,0.63,0.16,0.66,0.09,0.10,1.03,0.74,1.09,80.9,0.74,0.13,0.68,0.11,0.11,1.12,0.65,0.99,69.5,0.65,0.09,0.65,0.08,0.10,0.96,0.72,1.20,74.3,0.72,0.11,0.60,0.09,0.07,1.17,0.54,0.91,58.9,0.54,0.10,0.59,0.05,0.06,0.90,0.67,1.09,75.3,0.67,0.16,0.62,0.11,0.07,1.10,0.65,1.14,73.1,0.65,0.09,0.57,0.10,0.06,1.11,0.71,1.56,75.6,0.71,0.10,0.46,0.04,0.08,1.26,0.80,1.25,86.2,0.80,0.11,0.64,0.07,0.09,1.24
3,0.78,1.20,87.1,0.78,0.18,0.65,0.17,0.04,1.36,0.79,1.52,96.5,0.79,0.31,0.52,0.09,0.14,1.42,0.86,1.89,89.4,0.86,0.09,0.46,0.10,0.11,1.46,0.98,1.68,97.0,0.98,0.14,0.58,0.12,0.15,1.50,0.68,1.39,65.4,0.68,0

In [38]:
y = train_merged_df['who_win']
y

0      0
1      1
2      1
3      1
4      1
      ..
692    1
693    1
694    0
695    1
696    0
Name: who_win, Length: 697, dtype: int64

## Apply MinMaxScaler

In [39]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X.values), columns=X.columns, index=X.index)
X

,p1_kills_per_round_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p2_kills_per_round_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p3_kills_per_round_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p4_kills_per_round_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p5_kills_per_round_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p1_kills_per_round_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p2_kills_per_round_team_2,p2_kd_ratio_team_2,p2_damage_per_round_team_2,p2_kills_per_round_team_2,p2_assists_per_round_team_2,p2_deaths_per_round_team_2,p2_saved_by_teammate_per_round_team_2,p2_saved_teammates_per_round_team_2,p2_rating_team_2,p3_kills_per_round_team_2,p3_kd_ratio_team_2,p3_damage_per_round_team_2,p3_kills_per_round_team_2,p3_assists_per_round_team_2,p3_deaths_per_round_team_2,p3_saved_by_teammate_per_round_team_2,p3_saved_teammates_per_round_team_2,p3_rating_team_2,p4_kills_per_round_team_2,p4_kd_ratio_team_2,p4_damage_per_round_team_2,p4_kills_per_round_team_2,p4_assists_per_round_team_2,p4_deaths_per_round_team_2,p4_saved_by_teammate_per_round_team_2,p4_saved_teammates_per_round_team_2,p4_rating_team_2,p5_kills_per_round_team_2,p5_kd_ratio_team_2,p5_damage_per_round_team_2,p5_kills_per_round_team_2,p5_assists_per_round_team_2,p5_deaths_per_round_team_2,p5_saved_by_teammate_per_round_team_2,p5_saved_teammates_per_round_team_2,p5_rating_team_2
0,0.542373,0.420561,0.685393,0.542373,0.56250,0.8750,0.666667,0.315789,0.598765,0.644860,0.500000,0.718165,0.644860,0.419355,0.901235,0.388889,0.444444,0.664474,0.621359,0.534392,0.704186,0.621359,0.565217,0.741176,0.421053,0.50,0.656051,0.660377,0.539216,0.742743,0.660377,0.500000,0.741176,0.5625,0.380952,0.708609,0.84,0.685,0.828935,0.84,0.407407,0.677778,0.40,0.521739,0.745455,0.485030,0.318766,0.610892,0.485030,0.689655,0.750000,0.578947,0.833333,0.525490,0.752577,0.494465,0.679853,0.752577,0.366667,0.679012,0.368421,0.434783,0.783784,0.654206,0.590426,0.600000,0.654206,0.461538,0.7875,0.8125,0.380952,0.635838,0.595041,0.497585,0.725615,0.595041,0.538462,0.833333,0.636364,0.454545,0.664706,0.636364,0.577540,0.680729,0.636364,0.482759,0.7250,0.428571,0.321429,0.726027
1,0.516949,0.397196,0.675094,0.516949,0.46875,0.8875,0.500000,0.315789,0.580247,0.710280,0.664894,0.767790,0.710280,0.387097,0.740741,0.444444,0.555556,0.789474,0.621359,0.497354,0.675349,0.621359,0.608696,0.800000,0.526316,0.45,0.630573,0.660377,0.534314,0.782783,0.660377,0.653846,0.752941,0.4375,0.380952,0.721854,0.66,0.530,0.649071,0.66,0.333333,0.688889,0.35,0.347826,0.618182,0.455090,0.300771,0.564304,0.455090,0.551724,0.738636,0.526316,0.277778,0.474510,0.608247,0.346863,0.596136,0.608247,0.500000,0.765432,0.578947,0.347826,0.648649,0.700935,0.648936,0.651220,0.700935,0.423077,0.7625,0.3125,0.380952,0.676301,0.727273,0.864734,0.755697,0.727273,0.384615,0.583333,0.181818,0.590909,0.829412,0.565657,0.443850,0.659636,0.565657,0.586207,0.8375,0.523810,0.250000,0.65

## Train ANN

In [40]:
X.shape

(697, 90)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30/697, random_state=6)

In [42]:
X_train.shape

(666, 90)

In [43]:
X_test.shape

(31, 90)

In [44]:
no_inputs_X = X_train.shape[1]

In [45]:
model_ANN = keras.Sequential([
    keras.layers.Dense(8, activation='tanh', input_shape=(no_inputs_X,)),
    keras.layers.Dense(4, activation='tanh'),
    keras.layers.Dense(2, activation='tanh'),
    keras.layers.Dense(1, activation='sigmoid')
])


model_ANN.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [46]:
%%capture 
model_ANN.fit(X_train, y_train, epochs=600);

In [47]:
test_loss, test_accuracy = model_ANN.evaluate(X_train, y_train)

21/21 [==============================] - 0s 600us/step - loss: 0.5390 - accuracy: 0.7162


In [48]:
test_loss, test_accuracy = model_ANN.evaluate(X_test, y_test)

1/1 [==============================] - 0s 16ms/step - loss: 0.9485 - accuracy: 0.4839


## Classis MLs with crossval

In [49]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

models = [KNeighborsClassifier(n_neighbors=13), LinearDiscriminantAnalysis(), GaussianNB(), LogisticRegression(solver='liblinear', max_iter=1000)]
for model in models:
    scores = cross_val_score(model, X, y, cv=20)
    print(f'{model} mean score {round(scores.mean(),3)}, score std {round(scores.std(),4)}')


KNeighborsClassifier(n_neighbors=13) mean score 0.502, score std 0.0614
LinearDiscriminantAnalysis() mean score 0.505, score std 0.095
GaussianNB() mean score 0.484, score std 0.085
LogisticRegression(max_iter=1000, solver='liblinear') mean score 0.482, score std 0.0832


# Final analysis (3rd iteration)

Logistic regression tends to work better than ANNs.

## Inner merge of two tables in 2 steps

In [192]:
players_feats_3 = players_feats.drop(columns=['p1_id', 'p2_id', 'p3_id', 'p4_id', 'p5_id'])

In [193]:
# step 1 - merge with team 1
merged_df = pd.merge(train_data, players_feats_3, left_on = ['map_id', 'team1_id', 'map_name'], right_on =['map_id', 'team_id', 'map_name'])

In [194]:
# step 2 - merge with team 2
train_merged_df = pd.merge(merged_df, players_feats_3, left_on = ['map_id', 'team2_id', 'map_name'], right_on = ['map_id', 'team_id', 'map_name'], suffixes = ('_team_1', '_team_2'))
train_merged_df

,map_id,team1_id,team2_id,map_name,who_win,p1_total_kills_team_1,p1_headshots_team_1,p1_total_deaths_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_grenade_damage_per_round_team_1,p1_maps_played_team_1,p1_rounds_played_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p1_kill_death_team_1,p1_kill_round_team_1,p1_rounds_with_kills_team_1,p1_kill_death_difference_team_1,p1_total_opening_kills_team_1,p1_total_opening_deaths_team_1,p1_opening_kill_ratio_team_1,p1_opening_kill_rating_team_1,p1_team_win_percent_after_first_kill_team_1,p1_first_kill_in_won_rounds_team_1,p2_total_kills_team_1,p2_headshots_team_1,p2_total_deaths_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_grenade_damage_per_round_team_1,p2_maps_played_team_1,p2_rounds_played_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p2_kill_death_team_1,p2_kill_round_team_1,p2_rounds_with_kills_team_1,p2_kill_death_difference_team_1,p2_total_opening_kills_team_1,p2_total_opening_deaths_team_1,p2_opening_kill_ratio_team_1,p2_opening_kill_rating_team_1,p2_team_win_percent_after_first_kill_team_1,p2_first_kill_in_won_rounds_team_1,p3_total_kills_team_1,p3_headshots_team_1,p3_total_deaths_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_grenade_damage_per_round_team_1,p3_maps_played_team_1,p3_rounds_played_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p3_kill_death_team_1,p3_kill_round_team_1,p3_rounds_with_kills_team_1,p3_kill_death_difference_team_1,p3_total_opening_kills_team_1,p3_total_opening_deaths_team_1,p3_opening_kill_ratio_team_1,p3_opening_kill_rating_team_1,p3_team_win_percent_after_first_kill_team_1,p3_first_kill_in_won_rounds_team_1,p4_total_kills_team_1,p4_headshots_team_1,p4_total_deaths_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_grenade_damage_per_round_team_1,p4_maps_played_team_1,p4_rounds_played_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p4_kill_death_team_1,p4_kill_round_team_1,p4_rounds_with_kills_team_1,p4_kill_death_difference_team_1,p4_total_opening_kills_team_1,p4_total_opening_deaths_team_1,p4_opening_kill_ratio_team_1,p4_opening_kill_rating_team_1,p4_team_win_percent_after_first_kill_team_1,p4_first_kill_in_won_rounds_team_1,p5_total_kills_team_1,p5_headshots_team_1,p5_total_deaths_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_grenade_damage_per_round_team_1,p5_maps_played_team_1,p5_rounds_played_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p5_kill_death_team_1,p5_kill_round_team_1,p5_rounds_with_kills_team_1,p5_kill_death_difference_team_1,p5_total_opening_kills_team_1,p5_total_opening_deaths_team_1,p5_opening_kill_ratio_team_1,p5_opening_kill_rating_team_1,p5_team_win_percent_after_first_kill_team_1,p5_first_kill_in_won_rounds_team_1,team_id_team_1,p1_total_kills_team_2,p1_headshots_team_2,p1_total_deaths_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_grenade_damage_per_round_team_2,p1_maps_played_team_2,p1_rounds_played_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p1_kill_death_team_2,p1_kill_round_team_2,p1_rounds_with_kills_team_2,p1_kill_death_difference_team_2,p1_total_opening_kills_team_2,p1_total_opening_deaths_team_2,p1_opening_kill_ratio_team_2,p1_opening_kill_rating_team_2,p1_team_win_percent_after_first

## Prepare X and y for ANN

In [197]:
X = train_merged_df.drop(columns=['map_name','team1_id', 'team2_id','map_id','who_win','team_id_team_1', 'team_id_team_2'])
X

,p1_total_kills_team_1,p1_headshots_team_1,p1_total_deaths_team_1,p1_kd_ratio_team_1,p1_damage_per_round_team_1,p1_grenade_damage_per_round_team_1,p1_maps_played_team_1,p1_rounds_played_team_1,p1_kills_per_round_team_1,p1_assists_per_round_team_1,p1_deaths_per_round_team_1,p1_saved_by_teammate_per_round_team_1,p1_saved_teammates_per_round_team_1,p1_rating_team_1,p1_kill_death_team_1,p1_kill_round_team_1,p1_rounds_with_kills_team_1,p1_kill_death_difference_team_1,p1_total_opening_kills_team_1,p1_total_opening_deaths_team_1,p1_opening_kill_ratio_team_1,p1_opening_kill_rating_team_1,p1_team_win_percent_after_first_kill_team_1,p1_first_kill_in_won_rounds_team_1,p2_total_kills_team_1,p2_headshots_team_1,p2_total_deaths_team_1,p2_kd_ratio_team_1,p2_damage_per_round_team_1,p2_grenade_damage_per_round_team_1,p2_maps_played_team_1,p2_rounds_played_team_1,p2_kills_per_round_team_1,p2_assists_per_round_team_1,p2_deaths_per_round_team_1,p2_saved_by_teammate_per_round_team_1,p2_saved_teammates_per_round_team_1,p2_rating_team_1,p2_kill_death_team_1,p2_kill_round_team_1,p2_rounds_with_kills_team_1,p2_kill_death_difference_team_1,p2_total_opening_kills_team_1,p2_total_opening_deaths_team_1,p2_opening_kill_ratio_team_1,p2_opening_kill_rating_team_1,p2_team_win_percent_after_first_kill_team_1,p2_first_kill_in_won_rounds_team_1,p3_total_kills_team_1,p3_headshots_team_1,p3_total_deaths_team_1,p3_kd_ratio_team_1,p3_damage_per_round_team_1,p3_grenade_damage_per_round_team_1,p3_maps_played_team_1,p3_rounds_played_team_1,p3_kills_per_round_team_1,p3_assists_per_round_team_1,p3_deaths_per_round_team_1,p3_saved_by_teammate_per_round_team_1,p3_saved_teammates_per_round_team_1,p3_rating_team_1,p3_kill_death_team_1,p3_kill_round_team_1,p3_rounds_with_kills_team_1,p3_kill_death_difference_team_1,p3_total_opening_kills_team_1,p3_total_opening_deaths_team_1,p3_opening_kill_ratio_team_1,p3_opening_kill_rating_team_1,p3_team_win_percent_after_first_kill_team_1,p3_first_kill_in_won_rounds_team_1,p4_total_kills_team_1,p4_headshots_team_1,p4_total_deaths_team_1,p4_kd_ratio_team_1,p4_damage_per_round_team_1,p4_grenade_damage_per_round_team_1,p4_maps_played_team_1,p4_rounds_played_team_1,p4_kills_per_round_team_1,p4_assists_per_round_team_1,p4_deaths_per_round_team_1,p4_saved_by_teammate_per_round_team_1,p4_saved_teammates_per_round_team_1,p4_rating_team_1,p4_kill_death_team_1,p4_kill_round_team_1,p4_rounds_with_kills_team_1,p4_kill_death_difference_team_1,p4_total_opening_kills_team_1,p4_total_opening_deaths_team_1,p4_opening_kill_ratio_team_1,p4_opening_kill_rating_team_1,p4_team_win_percent_after_first_kill_team_1,p4_first_kill_in_won_rounds_team_1,p5_total_kills_team_1,p5_headshots_team_1,p5_total_deaths_team_1,p5_kd_ratio_team_1,p5_damage_per_round_team_1,p5_grenade_damage_per_round_team_1,p5_maps_played_team_1,p5_rounds_played_team_1,p5_kills_per_round_team_1,p5_assists_per_round_team_1,p5_deaths_per_round_team_1,p5_saved_by_teammate_per_round_team_1,p5_saved_teammates_per_round_team_1,p5_rating_team_1,p5_kill_death_team_1,p5_kill_round_team_1,p5_rounds_with_kills_team_1,p5_kill_death_difference_team_1,p5_total_opening_kills_team_1,p5_total_opening_deaths_team_1,p5_opening_kill_ratio_team_1,p5_opening_kill_rating_team_1,p5_team_win_percent_after_first_kill_team_1,p5_first_kill_in_won_rounds_team_1,p1_total_kills_team_2,p1_headshots_team_2,p1_total_deaths_team_2,p1_kd_ratio_team_2,p1_damage_per_round_team_2,p1_grenade_damage_per_round_team_2,p1_maps_played_team_2,p1_rounds_played_team_2,p1_kills_per_round_team_2,p1_assists_per_round_team_2,p1_deaths_per_round_team_2,p1_saved_by_teammate_per_round_team_2,p1_saved_teammates_per_round_team_2,p1_rating_team_2,p1_kill_death_team_2,p1_kill_round_team_2,p1_rounds_with_kills_team_2,p1_kill_death_difference_team_2,p1_total_opening_kills_team_2,p1_total_opening_deaths_team_2,p1_opening_kill_ratio_team_2,p1_opening_kill_rating_team_2,p1_team_win_percent_after_first_kill_team_2,p1_first_kill_in_won_rounds_team_2,p2_total_

X= pd.get_dummies(X, columns=['map_name','team1_id', 'team2_id'])

In [198]:
y = train_merged_df['who_win']
y

0      0
1      1
2      1
3      1
4      1
      ..
692    1
693    1
694    0
695    1
696    0
Name: who_win, Length: 697, dtype: int64

## Apply MinMaxScaler

In [199]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X.values), columns=X.columns, index=X.index)

In [200]:
from sklearn.linear_model import LogisticRegression

scores = cross_val_score(LogisticRegression(solver='liblinear', max_iter=1000, penalty='l2'), X, y, cv=20)
print(f'{model} mean score {round(scores.mean(),3)}, score std {round(scores.std(),4)}')
print(scores)


LogisticRegression(max_iter=1000, solver='liblinear') mean score 0.594, score std 0.0972
[0.65714286 0.45714286 0.68571429 0.62857143 0.4        0.6
 0.68571429 0.54285714 0.6        0.57142857 0.6        0.45714286
 0.68571429 0.6        0.45714286 0.54285714 0.8        0.64705882
 0.55882353 0.70588235]


In [201]:
from sklearn.ensemble import BaggingClassifier

logistic_ensemble = BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear', max_iter=1000, penalty='l2'),
                 n_estimators=30,
                 bootstrap=True,
                 max_samples=1,
                 max_features=0.7).fit(X,y)



In [202]:
logistic_ensemble.score(X,y)

0.45767575322812054

In [203]:
model = LogisticRegression(solver='liblinear', max_iter=1000, penalty='l2')
model.fit(X,y)
model.score(X,y)

0.7288378766140603

In [205]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

models = [KNeighborsClassifier(n_neighbors=13), LinearDiscriminantAnalysis(), GaussianNB(), LogisticRegression(solver='liblinear', max_iter=1000), RandomForestClassifier()]
for model in models:
    scores = cross_val_score(model, X, y, cv=20)
    print(f'{model} mean score {round(scores.mean(),3)}, score std {round(scores.std(),4)}')
    print(scores)


KNeighborsClassifier(n_neighbors=13) mean score 0.534, score std 0.08
[0.62857143 0.51428571 0.6        0.48571429 0.37142857 0.6
 0.45714286 0.48571429 0.51428571 0.51428571 0.6        0.37142857
 0.62857143 0.45714286 0.48571429 0.57142857 0.54285714 0.58823529
 0.64705882 0.61764706]
LinearDiscriminantAnalysis() mean score 0.563, score std 0.1007
[0.68571429 0.51428571 0.54285714 0.54285714 0.45714286 0.42857143
 0.71428571 0.31428571 0.54285714 0.54285714 0.57142857 0.65714286
 0.42857143 0.6        0.62857143 0.51428571 0.65714286 0.67647059
 0.55882353 0.67647059]
GaussianNB() mean score 0.519, score std 0.0707
[0.6        0.42857143 0.57142857 0.34285714 0.48571429 0.54285714
 0.45714286 0.57142857 0.57142857 0.57142857 0.6        0.51428571
 0.65714286 0.51428571 0.45714286 0.45714286 0.51428571 0.5
 0.47058824 0.55882353]
LogisticRegression(max_iter=1000, solver='liblinear') mean score 0.594, score std 0.0972
[0.65714286 0.45714286 0.68571429 0.62857143 0.4        0.6
 0.68571

In [247]:
X.shape

(697, 240)

In [363]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30/697, random_state=15)

In [364]:
X_train.shape

(666, 240)

In [365]:
X_test.shape

(31, 240)

In [366]:
no_inputs_X = X_train.shape[1]

In [367]:
model_ANN = keras.Sequential([
    keras.layers.Dense(3, activation='tanh', input_shape=(no_inputs_X,), kernel_regularizer=regularizers.l2(0.0001)),
    keras.layers.Dense(2, activation='tanh', kernel_regularizer=regularizers.l2(0.0001)),
    keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001))
])

model_ANN.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [368]:
%%capture 
model_ANN.fit(X_train, y_train, epochs=600);

In [369]:
test_loss, test_accuracy = model_ANN.evaluate(X_train, y_train)

21/21 [==============================] - 0s 650us/step - loss: 0.4172 - accuracy: 0.8514


In [370]:
test_loss, test_accuracy = model_ANN.evaluate(X_test, y_test)

1/1 [==============================] - 0s 15ms/step - loss: 0.7787 - accuracy: 0.6452


In [354]:
[0.6452, 0.5161, 0.6129, 0.4516, 0.5806, 0.6774, 0.6129]

[0.6452, 0.5161, 0.6129, 0.4516, 0.5806, 0.6774]